In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
    SELECT u.user_id,
           u.lt_day,
           CASE
               WHEN CAST(u.lt_day AS INTEGER) <= 365 THEN 'новый'
               ELSE 'старый'
            END AS is_new,
            CAST(u.age AS INTEGER) AS age,
            CASE
                WHEN u.gender_segment = 1.0 THEN 'женщина'
                WHEN u.gender_segment = 0.0 THEN 'мужчина'
            END AS gender_segment,
            u.os_name,
            u.cpe_type_name,
            l.country,
            l.city,
            SUBSTRING(ages.title, 4) AS age_segment,
            SUBSTRING(ts.title, 4) AS traffic_segment,
            SUBSTRING(ls.title, 4) AS lifetime_segment,
            u.nps_score,
            CASE
                WHEN u.nps_score >= 9 THEN 'сторонники'
                WHEN u.nps_score >= 7 THEN 'нейтралы'
                WHEN  u.nps_score < 7 AND u.nps_score >= 0 THEN 'критики'
            END AS nps_score_group
    FROM user AS u
    JOIN location AS l ON u.location_id=l.location_id
    JOIN age_segment AS ages ON u.age_gr_id=ages.age_gr_id
    JOIN traffic_segment AS ts ON u.tr_gr_id=ts.tr_gr_id
    JOIN lifetime_segment AS ls ON u.lt_gr_id=ls.lt_gr_id
    WHERE CAST(u.lt_day AS INTEGER) > 0 AND u.gender_segment IS NOT NULL;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_score_group
0,A001A2,2320,старый,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,старый,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,старый,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Дашборд на сайте Tableau Public:

https://public.tableau.com/views/sbproject2_2/sheet11?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

### Pdf-файл с презентацией:

https://disk.yandex.ru/i/ElEQHCWBOPzdgA